# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [100]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [101]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [102]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [103]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')


## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [104]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [105]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [106]:
gamma = 1
sigma = .005

In [107]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [108]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [109]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.5797 .


Pregunta 2:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 3:

Considere un swap a 1Y con cupones semestrales de 1.6% (lineal). Si la tasa
cupón cero a 0.5Y es 1.5% (exp), calcule la tasa cero a 1Y.


Pregunta 4:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 5:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.


Pregunta 6:

La fórmula para una opción sobre un bono c

## Respuestas

### Pregunta 1

In [110]:
#funcion obtenida del libro 12 con algunas alteraciones
def vasicek_path(r0: float,
                 gamma: float, r_: float,
                 sigma: float,
                 num_dias: int = 263,
                 dias_agno: int = 264,
                 pasos_dia: int = 1):
    """
    Retorna un camino de simulación del modelo de Vasicek.
    
    params:
    
    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día
    
    return:
    
    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (264)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    gamma_dt = gamma * dt
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        #r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * np.random.normal() # Discretización de Euler
        r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * -.5797
        result.append((i * dt, r))
    return result
sim = vasicek_path(curva(.0001), 1, 0.005, 0.005, num_dias=1)
sim=pd.DataFrame(sim)
print('Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón ceroentregada y número aleatorio N(0, 1) igual a -.5797 .')
print(sim)
print(f'Valor de la tasa = {sim.iloc[1,1]: 4%}')


Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón ceroentregada y número aleatorio N(0, 1) igual a -.5797 .
          0                      1
0  0.000000  0.0008062112176069023
1  0.003788            0.000643706
Valor de la tasa =  0.064371%


### Pregunta 2

In [111]:
k = .98
gamma=1
sigma=0.005
opcion = hw.zcb_call_put(
    hw.CallPut.CALL,
    k,
    curva(.0001),
    1,
    2,
    math.exp(-curva(365)),
    math.exp(-curva(730) * 730 / 365.0),
    gamma,
    sigma
)
print('Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5% valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%. Entregue el resultado 8 decimales y suponiendo un nocional de 1.')

print(f' Valor de la opcion = {opcion:.8f}')

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5% valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%. Entregue el resultado 8 decimales y suponiendo un nocional de 1.
 Valor de la opcion = 0.01955358


### Pregunta 3

In [128]:
print('Considere un swap a 1Y con cupones semestrales de 1.6% (lineal). Si la tasa cupón cero a 0.5Y es 1.5% (exp), calcule la tasa cero a 1Y.')

b=(1+0.016)**(2)-1
print(f'El valor de la tasa a 1Y es de {b: 2%}')

Considere un swap a 1Y con cupones semestrales de 1.6% (lineal). Si la tasa cupón cero a 0.5Y es 1.5% (exp), calcule la tasa cero a 1Y.
El valor de la tasa a 1Y es de  3.225600%


### Pregunta 4

La gran diferencia entre la simulacion de Hull-White en la medida libre de riesgo versus el
modelo de Hull-White en la medida T-forward es que cambia el parámetro de $\theta_t$ por el siguiente $\overline{\theta_t}$ dada las siguientes modificaciones:

$$
\overline{\theta_t}=\theta_t-B\left(t,T\right)\sigma^2
$$

In [112]:
print('Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el modelo de Hull-White en la medida T-forward.')

print('La gran diferencia entre la simulacion de Hull-White en la medida libre de riesgo versus el modelo de Hull-White en la medida T-forward es que cambia el parámetro del thetta por un thetta gorro, el cual se encuentra descrito anteriormente')

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el modelo de Hull-White en la medida T-forward.
La gran diferencia entre la simulacion de Hull-White en la medida libre de riesgo versus el modelo de Hull-White en la medida T-forward es que cambia el parámetro del thetta por un thetta gorro, el cual se encuentra descrito anteriormente


### Pregunta 5

La tasa forward lineal en $t$ que expira en $T$ y vence en $S$ con $S>T$ se denota con $F(t;T,S)$ y se define como:

$$
F(t;T,S):=\left(\frac{P(t,T)}{P(t,S)}-1\right)\cdot\frac{1}{S-T}=\left(\frac{W(t,S)}{W(t,T)}-1\right)\cdot\frac{1}{S-T}
$$

In [113]:
print('Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa forward entre 1282 y 2454 días.')
a = (curva(2454)/curva(1282)-1)*(1/(2454-1282))
print(f'valor de la tasa forward = {a: 4%}')

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa forward entre 1282 y 2454 días.
valor de la tasa forward =  0.200291%


### Pregunta 6

Verdadero, ya que en la valorización de una opción, tanto en el modelo Vasicek como en el modelo Hull-White, se usan exactamente los mismos parámetros, lo que cambia entre ambos modelos no es la fórmula cerrada, sinó que es en la simulación de montecarlo, donde cambia el $\gamma*\overline{r}$ por $\theta_t$, además, se cambia $A\left(t,T\right)$ dependiendo del modelo pero solo se usa para la valorización del bono cero cupon y no para valorizar la opción, además se mantiene el $B\left(t,T\right)$ 


In [114]:
print('La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es la misma que en el modelo de Hull-White. Comente.')

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es la misma que en el modelo de Hull-White. Comente.


### Pregunta 7

De esta forma la tasa utilizada para calcular el pago de un cupón entre las fechas $T > t$ de la pata flotante de un swap cámara promedio $(TNA)$ se determina a través de:

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

In [115]:
print('Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,020.00 (365 días después)')
TNA=round((10020/10000-1)*360/365,6)
#redondeo a 6 para tener más numeros que ceros, debiera ser a 4.
print(f'El valor de la TNA es de = {TNA}')
print(f'En extricto rivor, el valor de la TNA es de = {round(TNA,4)}')

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,020.00 (365 días después)
El valor de la TNA es de = 0.001973
En extricto rivor, el valor de la TNA es de = 0.002
